# Analyzing CIA Factbook Data Using SQL #
This project is purposed for learning SQL. The project consists of prompts and questions about data from the CIA World Factbook which are addressed or answered using SQL queries.

## Learning Concepts ##
Learning concepts included:
- SQL syntax,
- filtering return results using single criteria and multiple criteria (using both the AND and OR clauses),
- grouping, including grouping using AND and OR with parentheses,
- sorting results,
- creating summary statistics, using:
    - aggregate functions, including counts, minimum and maximum values, sums, and averages, and
    - arithmetic operations, with both integers and floats (and combinations of both),
- grouping,
- if/then logic,
- order of operations,
- the meaning of various SQL clauses (for example: `WHERE` filters results before an aggregation, wereas `HAVING` filters after aggregation),
- subqueries (or 'inner queries') in `WHERE` and `SELECT` statements, and
- returning rows that match a list of values.

## Dataset ##
The dataset is from the CIA World Factbook, a compendium of statistics about all countries. Here are the descriptions for some of the columns:
- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.
- `migration_rate` - the country's net migration rate, or the difference between the number of persons entering and leaving a country during the year per 1,000 persons.

---
sqlite is used for the database. pandas is used only to display the tables.

First, establish a connection to the database and get the database cursor.

In [233]:
import sqlite3
import pandas as pd

con = sqlite3.connect('./data_sets/factbook.db')
cur = con.cursor()

---
`execute_query` is a utility function for executing an SQL query on the dataset and returning the result as a pandas dataframe.

In [234]:
def execute_query(cursor, query):
    """ Execute query and return pandas dataframe.
    
    Keyword arguments:
    cursor - sqlite cursor
    query - sqlite query as string
    """
    query_result = cur.execute(query)
    cols = [column[0] for column in query_result.description]
    df = pd.DataFrame.from_records(data = query_result.fetchall(), columns = cols)
    
    return df

---
**Project instructions**: Write a query to return information on the tables in the database.

In [235]:
query = "SELECT * "\
        "FROM sqlite_master "\
        "WHERE type='table';"
df = execute_query(cur, query)
df

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


---
**Project instructions**: In a different code cell, write and run another query that returns the first five rows of the facts table in the database.

In [236]:
query = "SELECT * "\
        "FROM facts "\
        "LIMIT 5;"

df = execute_query(cur, query)
df

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


---
**Project instructions**: Write a single query that returns the following:
- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [237]:
query = "SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth) "\
        "FROM facts;"

df = execute_query(cur, query)
df

,MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,0,7256490011,0.0,4.02


---
**Project instructions**: Write a query that returns the countries with the minimum population.

In [238]:
sub_query = "SELECT MIN(population) "\
            "FROM facts"
    
query = "SELECT name "\
        "FROM facts "\
        f"WHERE population == ({sub_query});"

df = execute_query(cur, query)
df

,name
0,Antarctica


---
**Project instructions**: Write a query that returns the countries with the maximum population.

In [239]:
sub_query = "SELECT MAX(population) "\
            "FROM facts"

query = "SELECT name, printf('%,d', population) as 'population' "\
        "FROM facts "\
        f"WHERE population == ({sub_query});"

df = execute_query(cur, query)
df

,name,population
0,World,"7,256,490,011"


---
**Project instructions**: Recompute the summary statistics you found earlier while excluding the row for the whole world. Include the following:
- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [240]:
sub_query = "SELECT MAX(population) "\
            "FROM facts"

query = "SELECT MIN(population), "\
        "printf('%,d', MAX(population)) as 'MAX(population)', "\
        "MIN(population_growth), "\
        "MAX(population_growth) "\
        "FROM facts "\
        f"WHERE population < ({sub_query});"

df = execute_query(cur, query)
df

,MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,0,"1,367,485,388",0.0,4.02


---
**Project instructions**: In a different code cell, calculate the average value for the following columns:
- population
- area

In [241]:
query = "SELECT printf('%,d', AVG(population)) AS 'AVG(population)', "\
        "printf('%,d', AVG(area)) AS 'AVG(area)'"\
        "FROM facts;"

df = execute_query(cur, query)
df

,AVG(population),AVG(area)
0,"62,094,928","555,093"


---
**Project instructions**: Write a query that finds all countries meeting both of the following criteria:
- The population is above average.
- The area is below average.

In [242]:
sub_query1 = "SELECT AVG(population) "\
             "FROM facts"
sub_query2 = "SELECT AVG(area) "\
             "FROM facts"

query = "SELECT name, "\
        "printf('%,d', population) AS 'population', "\
        "printf('%,d', area) AS 'area', "\
        "printf('%,d', population / area) AS 'population_density'" \
        "FROM facts "\
        f"WHERE population > ({sub_query1}) AND area < ({sub_query2}) "\
        "ORDER BY (population / area) DESC;"

df = execute_query(cur, query)
df

,name,population,area,population_density
0,Bangladesh,"168,957,745","148,460","1,138"
1,Philippines,"100,998,376","300,000",336
2,Japan,"126,919,659","377,915",335
3,Vietnam,"94,348,835","331,210",284
4,United Kingdom,"64,088,222","243,610",263
5,Germany,"80,854,408","357,022",226
6,Thailand,"67,976,405","513,120",132


---
**Project question**: Which country has the most people? 

In [243]:
sub_query = "SELECT MAX(population) "\
            "FROM facts"

query = "SELECT name, "\
        "printf('%,d', MAX(population)) AS 'MAX(population)' "\
        "FROM facts "\
        f"WHERE population < ({sub_query});"

df = execute_query(cur, query)
df

,name,MAX(population)
0,China,"1,367,485,388"


---
**Project question**: Which country has the highest growth rate?

In [244]:
sub_query = "SELECT MAX(population_growth) "\
            "FROM facts"

query = "SELECT name, population_growth "\
        "FROM facts "\
        f"WHERE population_growth == ({sub_query});"

df = execute_query(cur, query)
df

,name,population_growth
0,South Sudan,4.02


---
**Project question**: Which countries have the highest ratios of water to land?

In [245]:
query = "SELECT name, CAST(area_water AS Float) / CAST(area_land AS Float) AS 'water_to_land_ratio' "\
        "FROM facts "\
        "ORDER BY water_to_land_ratio DESC "\
        "LIMIT 20;"

df = execute_query(cur, query)
df

,name,water_to_land_ratio
0,British Indian Ocean Territory,905.666667
1,Virgin Islands,4.520231
2,Puerto Rico,0.554791
3,"Bahamas, The",0.386613
4,Guinea-Bissau,0.284673
5,Malawi,0.259396
6,Netherlands,0.225710
7,Uganda,0.222922
8,Eritrea,0.164356
9,Liberia,0.156240


---
***Project question***: Which countries have more water than land?

In [246]:
query = "SELECT name, CAST(area_water AS Float) / CAST(area_land AS Float) AS 'water_to_land_ratio' "\
        "FROM facts "\
        "WHERE water_to_land_ratio > 1.0 "\
        "ORDER BY water_to_land_ratio DESC;"

df = execute_query(cur, query)
df

,name,water_to_land_ratio
0,British Indian Ocean Territory,905.666667
1,Virgin Islands,4.520231


---
**Project question**: Which countries will add the most people to their populations next year?

In [247]:
query = "SELECT name, population_growth, "\
        "CAST(population AS Float) * population_growth / 100.0 AS 'number_people_added' "\
        "FROM facts "\
        "ORDER BY number_people_added DESC "\
        "LIMIT 20;"

df = execute_query(cur, query)
df

,name,population_growth,number_people_added
0,World,1.08,7.837009e+07
1,India,1.22,1.527069e+07
2,China,0.45,6.153684e+06
3,Nigeria,2.45,4.448270e+06
4,Pakistan,1.46,2.906653e+06
5,Ethiopia,2.89,2.874562e+06
6,Bangladesh,1.60,2.703324e+06
7,United States,0.78,2.506677e+06
8,Indonesia,0.92,2.355142e+06
9,"Congo, Democratic Republic of the",2.45,1.944691e+06


---
**Project question**: Which countries have a higher death rate than birth rate?

In [248]:
query = "SELECT name, birth_rate, death_rate, (death_rate - birth_rate) AS lives_lost_rate "\
        "FROM facts "\
        "WHERE death_rate > birth_rate "\
        "ORDER BY lives_lost_rate DESC;"

df = execute_query(cur, query)
df

,name,birth_rate,death_rate,lives_lost_rate
0,Bulgaria,8.92,14.44,5.52
1,Serbia,9.08,13.66,4.58
2,Latvia,10.00,14.31,4.31
3,Lithuania,10.10,14.27,4.17
4,Ukraine,10.72,14.46,3.74
5,Hungary,9.16,12.73,3.57
6,Germany,8.47,11.42,2.95
7,Slovenia,8.42,11.37,2.95
8,Romania,9.14,11.90,2.76
9,Croatia,9.45,12.18,2.73


---
**Project question**: Which countries have the highest population/area ratio (i.e. population density)?  How does it compare to list of countries with population greater than average and area lower than average?

In [249]:
query = "SELECT name, "\
        "printf('%,d', population) AS 'population', "\
        "printf('%,d', area) AS 'area', "\
        "printf('%,d', population / area) AS 'population_density'" \
        "FROM facts "\
        "ORDER BY (population / area) DESC "\
        "LIMIT 30"

df = execute_query(cur, query)
df

,name,population,area,population_density
0,Macau,"592,731",28,"21,168"
1,Monaco,"30,535",2,"15,267"
2,Singapore,"5,674,472",697,"8,141"
3,Hong Kong,"7,141,106","1,108","6,445"
4,Gaza Strip,"1,869,055",360,"5,191"
5,Gibraltar,"29,258",6,"4,876"
6,Bahrain,"1,346,613",760,"1,771"
7,Maldives,"393,253",298,"1,319"
8,Malta,"413,965",316,"1,310"
9,Bermuda,"70,196",54,"1,299"


Bangladesh, with a population density of 1,138 people per square kilometer, is the only country in the top 30 countries with highest population densities, that also has a population greater than average and an area lower than average.

---
## Individual Exploration ##
Following are queries I created to further explore the data on my own.

---
Which countries have the highest *net gain* and *net loss* over the year? 

Calculate `rate_total` by adding `birth_rate` to `migration_rate` and subtracting `death_rate`. Calculate `net_growth` by dividing `rate_total` by 1000, and multiplying by `population`.

In [250]:
query = "SELECT name, population, population_growth, birth_rate, death_rate, migration_rate, "\
        "(birth_rate - death_rate + migration_rate) AS rate_total, "\
        "((birth_rate - death_rate + migration_rate) / 1000.0) * CAST(population As Float) AS 'net_growth' "\
        "FROM facts "\
        "WHERE name NOT IN ('World', 'European Union') "\
        "ORDER BY net_growth DESC "\
        "LIMIT 20;"

df = execute_query(cur, query)
df

,name,population,population_growth,birth_rate,death_rate,migration_rate,rate_total,net_growth
0,India,1251695584,1.22,19.55,7.32,0.04,12.27,1.535830e+07
1,China,1367485388,0.45,12.49,7.53,0.44,5.40,7.384421e+06
2,Nigeria,181562056,2.45,37.64,12.90,0.22,24.96,4.531789e+06
3,Pakistan,199085847,1.46,22.58,6.49,1.54,17.63,3.509883e+06
4,Indonesia,255993674,0.92,16.72,6.37,1.16,11.51,2.946487e+06
5,Ethiopia,99465819,2.89,37.27,8.19,0.22,29.30,2.914348e+06
6,Bangladesh,168957745,1.60,21.14,5.61,0.46,15.99,2.701634e+06
7,United States,321368864,0.78,12.49,8.15,3.86,8.20,2.635225e+06
8,Philippines,100998376,1.61,24.27,6.11,2.09,20.25,2.045217e+06
9,"Congo, Democratic Republic of the",79375136,2.45,34.88,10.07,0.27,25.08,1.990728e+06


In [251]:
query = "SELECT name, population, population_growth, birth_rate, death_rate, migration_rate, "\
        "(birth_rate - death_rate + migration_rate) AS rate_total, "\
        "((birth_rate - death_rate + IFNULL(migration_rate, 0)) / 1000.0) * CAST(population As Float) AS 'net_growth' "\
        "FROM facts "\
        "WHERE (name NOT IN ('World', 'European Union')) AND (net_growth < 0)"\
        "ORDER BY net_growth ASC "\
        "LIMIT 20;"

df = execute_query(cur, query)
df

,name,population,population_growth,birth_rate,death_rate,migration_rate,rate_total,net_growth
0,Japan,126919659,0.16,7.93,9.51,0.00,-1.58,-200533.06122
1,Germany,80854408,0.17,8.47,11.42,1.24,-1.71,-138261.03768
2,Ukraine,44429471,0.60,10.72,14.46,2.25,-1.49,-66199.91179
3,Russia,142423773,0.04,11.60,13.69,1.69,-0.40,-56969.50920
4,Romania,21666350,0.30,9.14,11.90,0.24,-2.52,-54599.20200
5,Bulgaria,7186893,0.58,8.92,14.44,0.29,-5.23,-37587.45039
6,Serbia,7176794,0.46,9.08,13.66,0.00,-4.58,-32869.71652
7,Hungary,9897541,0.22,9.16,12.73,1.33,-2.24,-22170.49184
8,Belarus,9589689,0.20,10.70,13.36,0.70,-1.96,-18795.79044
9,Croatia,4464844,0.13,9.45,12.18,1.39,-1.34,-5982.89096


*Note*: `population_growth`, defined as 'the country's population growth as a percentage', should presumably correspond with `rate_total` but it does not.

---
Which countries have the highest *rate of gain* and *rate of loss* over the year? 

(see above for calculation of `rate_total`)

In [252]:
query = "SELECT name, population, population_growth, birth_rate, death_rate, migration_rate, "\
        "(birth_rate - death_rate + migration_rate) AS rate_total, "\
        "((birth_rate - death_rate + migration_rate) / 1000.0) * CAST(population As Float) AS 'net_growth' "\
        "FROM facts "\
        "WHERE name NOT IN ('World', 'European Union') "\
        "ORDER BY rate_total DESC "\
        "LIMIT 20;"

df = execute_query(cur, query)
df

,name,population,population_growth,birth_rate,death_rate,migration_rate,rate_total,net_growth
0,South Sudan,12042910,4.02,36.91,8.18,11.47,40.20,4.841250e+05
1,American Samoa,54343,0.30,22.89,4.75,21.13,39.27,2.134050e+03
2,Syria,17064854,0.16,22.17,4.00,19.79,37.96,6.477819e+05
3,"Micronesia, Federated States of",105216,0.46,20.54,4.23,20.93,37.24,3.918244e+03
4,Tonga,106501,0.03,23.00,4.85,17.84,35.99,3.832971e+03
5,Sao Tome and Principe,194006,1.84,34.23,7.24,8.63,35.62,6.910494e+03
6,Somalia,10616380,1.83,40.45,13.62,8.49,35.32,3.749705e+05
7,Jordan,8117564,0.83,25.37,3.79,13.24,34.82,2.826536e+05
8,Mali,16955536,2.98,44.99,12.89,2.26,34.36,5.825922e+05
9,Uganda,37101745,3.24,43.79,10.69,0.74,33.84,1.255523e+06


In [253]:
query = "SELECT name, population, population_growth, birth_rate, death_rate, migration_rate, "\
        "(birth_rate - death_rate + migration_rate) AS rate_total, "\
        "((birth_rate - death_rate + IFNULL(migration_rate, 0)) / 1000.0) * CAST(population As Float) AS 'net_growth' "\
        "FROM facts "\
        "WHERE (name NOT IN ('World', 'European Union')) AND (rate_total < 0)"\
        "ORDER BY rate_total ASC "\
        "LIMIT 20;"

df = execute_query(cur, query)
df

,name,population,population_growth,birth_rate,death_rate,migration_rate,rate_total,net_growth
0,Bulgaria,7186893,0.58,8.92,14.44,0.29,-5.23,-37587.45039
1,Serbia,7176794,0.46,9.08,13.66,0.00,-4.58,-32869.71652
2,Slovenia,1983412,0.26,8.42,11.37,0.37,-2.58,-5117.20296
3,Romania,21666350,0.30,9.14,11.90,0.24,-2.52,-54599.20200
4,Hungary,9897541,0.22,9.16,12.73,1.33,-2.24,-22170.49184
5,Belarus,9589689,0.20,10.70,13.36,0.70,-1.96,-18795.79044
6,Germany,80854408,0.17,8.47,11.42,1.24,-1.71,-138261.03768
7,Japan,126919659,0.16,7.93,9.51,0.00,-1.58,-200533.06122
8,Ukraine,44429471,0.60,10.72,14.46,2.25,-1.49,-66199.91179
9,Croatia,4464844,0.13,9.45,12.18,1.39,-1.34,-5982.89096


---
Find all countries meeting both of the following criteria:
- The population density is above average.
- The net_growth is above average.

In [254]:
sub_query1 = "SELECT AVG(population / area) "\
             "FROM facts"
sub_query2 = "SELECT AVG(((birth_rate - death_rate + IFNULL(migration_rate, 0)) / 1000.0) * CAST(population As Float)) "\
             "FROM facts"

query = "SELECT name, area, population, CAST(population AS Float) / CAST(area AS Float) AS 'population_density', "\
        "((birth_rate - death_rate + IFNULL(migration_rate, 0)) / 1000.0) * CAST(population As Float) AS 'net_growth' "\
        "FROM facts "\
        "WHERE (name NOT IN ('World', 'European Union')) "\
        f"AND (population_density > ({sub_query1})) AND (net_growth > ({sub_query2}));"

df = execute_query(cur, query)
df

,name,area,population,population_density,net_growth
0,Bangladesh,148460,168957745,1138.069143,2.701634e+06
